In [ ]:
# Starta med covering sets
# vi börjar enkelt med 5 matcher
# Eftersom vi tänker oss att vi delar in grupper av matcher
# Speciellt om vi är osäkra på utfallet för vissa, då kanske vi vill garantera

In [1]:
import itertools



In [62]:
#----------------------------------------------------
# Utvärdera hur stor chans vi har att vinna!
#----------------------------------------------------
# Som om varje enskilt utfall vore lika sannolikt...
# 
# Räkna hur många i all_combinations som får korrekt mha covering set...
# 

import itertools
from collections import Counter

def analyze_covering_set(covering_set, all_combinations):
    correct_counts = Counter()

    for combo in all_combinations:
        max_correct = 0
        for ticket in covering_set:
            correct = sum(a == b for a, b in zip(ticket, combo))
            max_correct = max(max_correct, correct)
        correct_counts[max_correct] += 1

    result = correct_counts
    for correct, count in sorted(result.items()):
        print(f"Combinations with {correct} correct: {count} {round(100*count/len(all_combinations), 2)}% ")

    return "Finished."

# Example usage
# covering_set = [
#     ('1', '1', '1', '1', '1'),
#     ('X', 'X', 'X', 'X', 'X'),
#     ('2', '2', '2', '2', '2'),
#     ('1', '1', '1', 'X', '2'),
#     ('X', 'X', '1', '2', '2')
# ]
# Vi använder covering_set från ovan istället!

# Generera alla kombinationer möjliga på 5 matcher
## all_combinations = list(itertools.product('1X2', repeat=5))
# Använd all combinations från ovan!

#result = analyze_covering_set(covering_set, all_combinations)

# Display the results
#for correct, count in sorted(result.items()):
#    print(f"Combinations with {correct} correct: {count} {round(100*count/len(all_combinations), 2)}% ")


In [60]:
# Fall nummer 1 : Det helt naiva sättet - 
# Vi räknar ut hur många av M matcher vi får 4 rätt på.

covering_set = []
all_combinations = list(itertools.product('1X2', repeat=5))

covered_combinations = set()

while len(covered_combinations) < len(all_combinations):
    best_ticket = None
    max_new_covered = 0
    
    # Generate a candidate ticket
    for candidate in all_combinations:
        new_covered = sum(1 for combo in all_combinations 
                          if combo not in covered_combinations 
                          and sum(a == b for a, b in zip(candidate, combo)) >= 4)
        
        if new_covered > max_new_covered:
            max_new_covered = new_covered
            best_ticket = candidate
    
    if best_ticket:
        covering_set.append(best_ticket)
        for combo in all_combinations:
            if sum(a == b for a, b in zip(best_ticket, combo)) >= 4:
                covered_combinations.add(combo)


In [61]:
# Utvärdera

In [63]:
analyze_covering_set(covering_set, all_combinations)

Combinations with 4 correct: 212 87.24% 
Combinations with 5 correct: 31 12.76% 


'Finished.'

In [ ]:
# Om vi har vissa utfall omöjliga så måste vi berätta det

In [64]:
# Fall nummer 2 : ta bort vissa utfall från några av matcherna.
# Generera rader som täcker in n av M möjliga.
# constrained = färre möjliga rader tas in från början.
# 

import itertools

def generate_constrained_combinations(game_constraints):
    return list(itertools.product(*[constraints for constraints in game_constraints]))

def generate_covering_set(game_constraints, guarantee):
    all_combinations = generate_constrained_combinations(game_constraints)
    covering_set = []
    covered_combinations = set()

    while len(covered_combinations) < len(all_combinations):
        best_ticket = None
        max_new_covered = 0
        # new_covered --> antal träffar som en ticket har som bäst mot alla kombinationer... men minus redan täckta kombinationer.
        for candidate in all_combinations:
            new_covered = sum(1 for combo in all_combinations 
                              if combo not in covered_combinations 
                              and sum(a == b for a, b in zip(candidate, combo)) >= guarantee)
            
            if new_covered > max_new_covered:
                max_new_covered = new_covered
                best_ticket = candidate

        # Behåll alla tickets som täcker tillräckligt mycket - dessa behöver vi inte jämföra mot igen...
        if best_ticket:
            covering_set.append(best_ticket)
            for combo in all_combinations:
                if sum(a == b for a, b in zip(best_ticket, combo)) >= guarantee:
                    covered_combinations.add(combo)
        else:
            break  # No more tickets can improve coverage

    return covering_set

# Example usage
game_constraints = [
    ('1', '2'),  # Game 1: Only 1 and 2 are possible
    ('1', 'X', '2'),  # Game 2: All outcomes possible
    ('1', 'X', '2'),  # Game 3: All outcomes possible
    ('1', 'X', '2'),  # Game 4: All outcomes possible
    ('1', 'X', '2'),  # Game 4: All outcomes possible
    ('1', 'X', '2'),  # Game 4: All outcomes possible
    ('1', 'X')   #
]
all_combinations =  generate_constrained_combinations(game_constraints)



covering_set = generate_covering_set(game_constraints, 6)
print(f"Number of tickets in covering set: {len(covering_set)}")
for ticket in covering_set[:5]:
    print(ticket)

analyze_covering_set(covering_set, all_combinations)


Number of tickets in covering set: 118
('1', '1', '1', '1', '1', '1', '1')
('1', '1', '1', '1', 'X', 'X', 'X')
('1', '1', '1', 'X', '1', '2', 'X')
('1', '1', '1', 'X', '2', 'X', '1')
('1', '1', '1', '2', 'X', '2', '1')
Combinations with 6 correct: 854 87.86% 
Combinations with 7 correct: 118 12.14% 


'Finished.'

In [56]:
all_combinations =  generate_constrained_combinations(game_constraints)


In [58]:
len(all_combinations)

972

In [48]:
print(123+15+87+18)

243


In [ ]:
import random
from collections import Counter

# Function to generate outcome combinations based on individual match probabilities.
def generate_probable_outcomes(match_probabilities, num_samples=10000):
    # Each element in match_probabilities is a tuple specifying weights for ['1', 'X', '2']
    outcomes = []
    num_matches = len(match_probabilities)
    for _ in range(num_samples):
        outcome = tuple(random.choices(['1', 'X', '2'], weights=match_probabilities[i], k=1)[0]
                        for i in range(num_matches))
        outcomes.append(outcome)
    # Optionally deduplicate if many repeated outcomes are expected
    return list(set(outcomes))

# Greedy algorithm for building the covering set.
def generate_covering_set(probable_combinations, guarantee):
    covering_set = []
    covered_combinations = set()
    
    # Continue until every outcome in the probable sample is covered.
    while len(covered_combinations) < len(probable_combinations):
        best_ticket = None
        max_new_covered = 0
        
        # Evaluate each candidate ticket from the probable outcomes
        for candidate in probable_combinations:
            new_covered = sum(
                1 for outcome in probable_combinations
                if outcome not in covered_combinations and sum(a == b for a, b in zip(candidate, outcome)) >= guarantee
            )
            
            if new_covered > max_new_covered:
                max_new_covered = new_covered
                best_ticket = candidate
        
        if best_ticket:
            covering_set.append(best_ticket)
            for outcome in probable_combinations:
                if sum(a == b for a, b in zip(best_ticket, outcome)) >= guarantee:
                    covered_combinations.add(outcome)
        else:
            break  # No candidate can add new coverage; exit loop.
    return covering_set

# Example usage:
# Assume each match has probabilities. For example, for 15 matches, you may have:
# Each tuple corresponds to the weights for outcomes '1', 'X', '2' respectively.

match_odds = [
    (2.02, 3.18, 4.35), 
    (2.07 , 4.02 , 3.25 ), 
    (1.44, 4.76 , 7.07 ), 
    ( 2.16, 3.43 , 3.50 ), 
    ( 2.38, 3.65 , 2. ), 
    ( ,  ,  ), 
    ( ,  ,  ), 
    ( ,  ,  ), 
    ( ,  ,  ), 
    ( ,  ,  ), 
    ( ,  ,  ), 
    ( ,  ,  ), 
    ( ,  ,  ), 
    
]
match_probabilities = [
    (0.35, 0.26, 0.39),  # Match 1
    (0.6, 0.25, 0.15), # Match 2
    # ... continue for all 15 matches
]
# For illustration, here we assume 5 matches. In your actual case, you may have more.
# Generate the 10,000 most probable outcomes.
probable_combinations = generate_probable_outcomes(match_probabilities, num_samples=10000)

# Build the covering set with a guarantee of, say, 13 correct predictions.
covering_set = generate_covering_set(probable_combinations, guarantee=13)

print("Number of tickets in covering set:", len(covering_set))
# Optionally, analyze how well the covering set works against the generated probable outcomes.
